## Goal: update master table with fixed population-sharing values 

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import upsetplot
import os
import matplotlib.pyplot as plt
import yaml
from snakemake.io import expand
import pyranges as pr
from pyfaidx import Fasta
from mizani.formatters import percent_format
from scipy import stats


p = os.path.dirname(os.getcwd())
sys.path.append(p)

from scripts.utils import *
from scripts.vcf_utils import *
from scripts.plotting import *

from plotnine import *

In [2]:
config = load_config()
od = '../'

def proc_cfg(entry, od):
    entry = entry.replace('../../', '')
    entry = od+entry
    return entry

In [4]:
mt_df = pd.read_csv('../data/05_mastertable/26062025_PODER_mastertable.tsv', sep='\t')
# mt_df = mt_df.loc[mt_df['filter']=='pass']
pops = get_population_colors()[1]
mt_df['population_sharing_2'] = (mt_df[pops]>0).sum(axis=1)
mt_df['pop_spec_t'] = (mt_df.population_sharing==1)&\
                      (mt_df.sample_sharing>1)
mt_df['det_pop'] = mt_df[get_population_colors()[1]].idxmax(axis=1)

In [5]:
mt_df.loc[mt_df.population_sharing!=mt_df.population_sharing_2]

,isoform,geneid.v,associated_gene_biotype,proteinv47_Chromosome,proteinv47_Start,proteinv47_Stop,proteinv47_Strand,proteinv47_Source,proteinv47_CDS_Source,proteinv47_CDS_Start,...,perc_A_downstream_TTS,seq_A_downstream_TTS,ORF_seq,annotated,predicted_ORF,proteinv47_predicted_ORF,associated_gene_biotype_sub,population_sharing_2,pop_spec_t,det_pop


In [6]:
# read in the mt that is on zenodo
df = pd.read_csv('04_poder_mt.tsv', sep='\t')
df.to_csv('04_poder_mt_back_250721.tsv', sep='\t')

In [7]:
df.drop('population_sharing', axis=1, inplace=True)

In [10]:
df = df.merge(mt_df[['isoform', 'population_sharing']], how='left', on='isoform')
assert len(df.loc[df.population_sharing.isnull()].index) == 0

In [11]:
df.to_csv('04_poder_mt.tsv', sep='\t')

In [13]:
df.loc[df.population_sharing>df.sample_sharing]

,isoform,geneid.v,associated_gene_biotype,proteinv47_Start,proteinv47_Stop,proteinv47_CDS_Source,proteinv47_CDS_Start,proteinv47_CDS_Stop,proteinv47_pid,proteinv47_blastp_identity,...,FSM_class,perc_A_downstream_TTS,seq_A_downstream_TTS,pop_spec_t,trunc_annot_aa,elong_annot_aa,annot_aa,aa_novelty,tau,population_sharing


In [14]:
df['population_sharing_2'] = (df[pops]>0).sum(axis=1)
df.loc[df.population_sharing!=df.population_sharing_2]

,isoform,geneid.v,associated_gene_biotype,proteinv47_Start,proteinv47_Stop,proteinv47_CDS_Source,proteinv47_CDS_Start,proteinv47_CDS_Stop,proteinv47_pid,proteinv47_blastp_identity,...,perc_A_downstream_TTS,seq_A_downstream_TTS,pop_spec_t,trunc_annot_aa,elong_annot_aa,annot_aa,aa_novelty,tau,population_sharing,population_sharing_2


## also check the UMA master table

In [17]:
df = pd.read_csv('02_uma_mt.tsv', sep='\t')

In [18]:
df['population_sharing_2'] = (df[pops]>0).sum(axis=1)
df.loc[df.population_sharing!=df.population_sharing_2]

,associated_transcriptid.v,associated_geneid.v,isoform,flair_total_counts,flair_mean_counts,flair_expressed_samples,flair_min_counts,flair_max_counts,associated_transcript_biotype,associated_gene_biotype,...,start,end,strand,discovered_transcripts_per_gene,old_associated_geneid.v,old_associated_gene_biotype,existsFSMinGene,existsFSMinTranscript,filter,population_sharing_2
